In [11]:
from pylab import plot,show
import numpy as np
import pandas as pd
from numpy import linalg as LA
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import spsolve_triangular
from scipy.sparse import diags
plt.style.use('seaborn-white')

# Acoustic Wave Equation in Time Domain

The elastic wave equation can be stated as follows,
\begin{equation}
\frac{\partial^2 u}{\partial t^2}=c^2 \nabla^2 u+f,\qquad x\in \Omega \subset \mathbb{R}^d,~t\in(T_1,T_2]
\qquad\qquad\qquad\qquad (1)
\end{equation}
where $\Delta$ is *the Laplacian*, $f$ is a *forcing function* (for example our source) and $c$ is *the wave velocity* at which the time and spatially varying wave $u$ propagates.

# A fourth-Order Finite-Difference Method for the One Dimensional Wave Equation

A higher-order scheme can be introduced as follows (the readers are encouraged to see [[Liao, 2008](https://www.sciencedirect.com/science/article/abs/pii/S0096300308007042)]),
\begin{align}
\label{eq1A5.01}\left(1+\frac{1^2}{12}\delta_t^2\right)^{-1}\frac{\delta_t^2}{\tau^2} u_{i}^{n}=
c^2_i \left(1+\frac{1}{12}\delta_x^2\right)^{-1}\frac{\delta_x^2}{h^2} u_{i}^{n}+f_{i}^{n},
\end{align}
where it will be shown that this scheme is 4th-order.

In this section, the scheme is analyzed and also an numerical algorithm is generated. This equation can be also written in the following form,
\begin{align}\label{eq1A5.02}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2_i}=
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}
+\left(1+\frac{1}{12}\delta_t^2\right)\left(1+\frac{1}{12}\delta_x^2\right)\frac{f_{i}^{n}}{c^2_i}.
\qquad \qquad (13)
\end{align}
The left hand side of the equation (13) can be expanded as follows,
\begin{align}\label{eq1A5.03}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2_i}&=
\frac{1}{12\tau^2}\left[ \left(\frac{u_{i+1}^{n+1}}{c^2_{i+1}}+10 \frac{u_{i}^{n+1}}{c^2_{i}} + \frac{u_{i-1}^{n+1}}{c^2_{i-1}}\right)
-2\left(\frac{u_{i+1}^{n}}{c^2_{i+1}}+10 \frac{u_{i}^{n}}{c^2_{i}} + \frac{u_{i-1}^{n}}{c^2_{i-1}}\right)\right.\notag\\
&+\left. \left(\frac{u_{i+1}^{n-1}}{c^2_{i+1}}+10 \frac{u_{i}^{n-1}}{c^2_{i}} + \frac{u_{i-1}^{n-1}}{c^2_{i-1}}\right)\right].
\end{align}

It follows that,
\begin{align}
\label{eq1A5.07}
&\left[\frac{u_{i+1}^{n+1}}{c^2_{i+1}}+10 \frac{u_{i}^{n+1}}{c^2_{i}} + \frac{u_{i-1}^{n+1}}{c^2_{i-1}}\right]
-2 \left[\frac{u_{i+1}^{n}}{c^2_{i+1}}+10 \frac{u_{i}^{n}}{c^2_{i}} + \frac{u_{i-1}^{n}}{c^2_{i-1}}\right]+\left[\frac{u_{i+1}^{n-1}}{c^2_{i+1}}+10 \frac{u_{i}^{n-1}}{c^2_{i}} + \frac{u_{i-1}^{n-1}}{c^2_{i-1}}\right]
\notag\\ &
=\lambda^2\left[u_{i+1}^{n+1}-2 u_{i}^{n+1} + u_{i-1}^{n+1}\right]+10\lambda^2\left[u_{i+1}^{n}-2u_{i}^{n} + u_{i-1}^{n}\right]
+\lambda^2 \left[u_{i+1}^{n-1}-2u_{i}^{n-1} + u_{i-1}^{n-1}\right]
\notag\\ &
+\tau^2\mathbf{F}_{i}^{n}.
\qquad \qquad (14)
\end{align}
where
\begin{align}
\label{eq1A5.06}\mathbf{F}_{i}^{n}&=
\frac{1}{144}\left[\frac{1}{c^2_{i+1}}\left(f_{i+1}^{n+1}+10 f_{i+1}^{n}+f_{i+1}^{n-1}\right)
+\frac{10}{c^2_{i}}(f_{i}^{n+1}+10f_{i}^{n}+f_{i}^{n-1})\right.\notag\\
&+\left.\frac{1}{c^2_{i-1}}(f_{i-1}^{n+1}+10f_{i-1}^{n}+f_{i-1}^{n-1})\right]
\end{align}
The equation (14) can be simplified as follows,

\begin{align}
\left(\frac{1}{c^2_{i+1}}-\lambda^2\right)u_{i+1}^{n+1}+\left(\frac{10}{c^2_{i}}+2\lambda^2\right)u_{i}^{n+1}+\left(\frac{1}{c^2_{i-1}}-\lambda^2\right)u_{i-1}^{n+1}
&=
\left(\frac{2}{c^2_{i+1}}+10\lambda^2\right)u_{i+1}^{n}+\left(\frac{20}{c^2_{i}}-20\lambda^2\right)u_{i}^{n}
+\left(\frac{2}{c^2_{i-1}}+10\lambda^2\right)u_{i-1}^{n}
\notag \\ &
-\left(\frac{1}{c^2_{i+1}}-\lambda^2\right)u_{i+1}^{n-1}
-\left(\frac{10}{c^2_{i}}+2\lambda^2\right)u_{i}^{n-1}-\left(\frac{1}{c^2_{i-1}}-\lambda^2\right)u_{i-1}^{n-1}
+\tau^2\mathbf{F}_{i}^{n}.
\end{align}

Consider the following matrices,
$$
A=\begin{bmatrix}
\frac{10}{c^2_{1}}+2\lambda^2 & \frac{1}{c^2_{2}}-\lambda^2 & 0 & 0 &\dots & \dots &0 \\
\frac{1}{c^2_{1}}-\lambda^2 & \frac{10}{c^2_{2}}+2\lambda^2 & \frac{1}{c^2_{3}}-\lambda^2 & 0 &  0 &\dots &\vdots \\
0 & \frac{1}{c^2_{2}}-\lambda^2 & \frac{10}{c^2_{3}}+2\lambda^2 & \frac{1}{c^2_{4}}-\lambda^2 & 0 & \dots &\vdots \\
0 & 0& \ddots & \ddots & \ddots & 0 & \vdots \\
\vdots & \dots &0 & \frac{1}{c^2_{M-4}}-\lambda^2 & \frac{10}{c^2_{M-3}}+2\lambda^2 & \frac{1}{c^2_{N_x-2}}-\lambda^2 & 0\\
\vdots & \dots & 0 &0 & \frac{1}{c^2_{M-3}}-\lambda^2 & \frac{10}{c^2_{N_x-2}}+2\lambda^2 & \frac{1}{c^2_{N_x-1}}-\lambda^2 \\
0 & \dots & \dots & 0 &0 & \frac{1}{c^2_{N_x-2}}-\lambda^2 & \frac{10}{c^2_{N_x-1}}+2\lambda^2\\
\end{bmatrix},
$$
$$
B=\begin{bmatrix}
\frac{20}{c^2_{1}}-20\lambda^2 & \frac{2}{c^2_{2}}+10\lambda^2 & 0 & 0 &\dots & \dots &0 \\
\frac{2}{c^2_{1}}+10\lambda^2 & \frac{20}{c^2_{2}}-20\lambda^2 & \frac{2}{c^2_{3}}+10\lambda^2 & 0 &  0 &\dots &\vdots \\
0 & \frac{2}{c^2_{2}}+10\lambda^2 & \frac{20}{c^2_{3}}-20\lambda^2 & \frac{2}{c^2_{4}}+10\lambda^2 & 0 & 0 &\vdots \\
0 & 0& \ddots & \ddots & \ddots & 0 & \vdots \\
\vdots & \dots &0 & \frac{2}{c^2_{M-4}}+10\lambda^2 & \frac{20}{c^2_{M-3}}-20\lambda^2 & \frac{2}{c^2_{N_x-2}}+10\lambda^2 & 0\\
\vdots & \dots & 0 &0 & \frac{2}{c^2_{M-3}}+10\lambda^2 & \frac{20}{c^2_{N_x-2}}-20\lambda^2 & \frac{2}{c^2_{N_x-1}}+10\lambda^2 \\
0 & \dots & \dots & 0 &0 & \frac{2}{c^2_{N_x-2}}+10\lambda^2 & \frac{20}{c^2_{N_x-1}}-20\lambda^2\\
\end{bmatrix},
$$
$$
C=-A
$$
and also
$$
\textbf{u}^n=\begin{bmatrix}
u_{1}^{n} \\
u_{2}^{n}\\
\vdots\\
u_{N_x-2}^{n}\\
u_{N_x-1}^{n}
\end{bmatrix},~
\textbf{b}^n=
\begin{bmatrix}
-\left(\frac{1}{c^2_{0}}-\lambda^2\right)f_{a}^{n+1}+\left(\frac{2}{c^2_{0}}+10\lambda^2\right)f_{a}^{n}
-\left(\frac{1}{c^2_{0}}-\lambda^2\right)f_{a}^{n-1}\\
0\\
\vdots\\
0\\
-\left(\frac{1}{c^2_{N_x}}-\lambda^2\right)f_{b}^{n+1}+\left(\frac{2}{c^2_{N_x}}+10\lambda^2\right)f_{b}^{n}
-\left(\frac{1}{c^2_{0}}-\lambda^2\right)f_{b}^{n-1} \\
\end{bmatrix},
 \textbf{F}^n=\tau^2
\begin{bmatrix}
\mathbf{F}_{1}^{n}\\
\mathbf{F}_{2}^{n}\\
\vdots\\
\mathbf{F}_{N_x-2}^{n}\\
\mathbf{F}_{N_x-1}^{n}
\end{bmatrix}.
$$
Therefore, the presented scheme can be expressed in the following matrix form,
\begin{align}
\label{eq1A5.11}\textbf{u}^{n+1}&=A^{-1}\left(B \textbf{u}^{n}+C \textbf{u}^{n-1}+\textbf{b}^n+ \textbf{F}^n\right).
\end{align}

**Theorem (Stability fo the fourth-order scheme)**
Assume that the solution of the two dimensional acoustic wave equation is sufficiently smooth.
The stability criteria of the numerical scheme can be derived as follows,
\begin{equation}
\label{eq.1D4th-Stbl}
\lambda \max_{x\in \Omega} \{|c(x)|\}=c\lambda < 1.
\end{equation}

First off, without loss of generality, assume that $f_{i}^n=0$ for all values of $i$ and $n$. Moreover, let $c=\max_{(x)\in \Omega} \{|c(x)|\}$. It follows that,
\begin{align}\label{eq1A7.01}
\left(1+ \frac{1-c^2\lambda^2}{12}\delta_{x}^{2} \right) \delta_{t}^{2} u_{m,j}^{n}&= c^2\lambda^2 \delta_{x}^{2} u_{m,j}^{n}.
\qquad \qquad (15)
\end{align}
Considering $\rho=\frac{1-c^2\lambda^2}{12}$, it follows from (15)
\begin{align}\label{eq1A7.02}
&\left(1+\rho \delta_x^2\right)
\left[ u_{i,j}^{n+1}+u_{i,j}^{n-1}\right]
-\left[
2\left(1+\rho \delta_x^2\right)
+c^2\lambda^2 \delta_x^2 \right]u_{m,j}^{n}=0.
\qquad \qquad (16)
\end{align}
Applying discrete Fourier transformation on (16) leads to the following result,
\begin{align}\label{eq1A7.03}
&\left(1+\rho \delta_x^2\right)e^{ih\left(m \xi\right)}
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]-
\left[2\left(1+\rho \delta_x^2\right)+c^2\lambda^2 \delta_x^2\right] e^{ih\left(m \xi\right)} \hat{u}^{n}=0,
\end{align}
which also can be expressed as follows, considering $\theta=h \xi$,
\begin{align}\label{eq1A7.04}
&\left(1+\rho \delta_x^2\right)e^{im \theta}\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
-\left[2\left(1+\rho \delta_x^2\right)e^{im \theta}
+c^2\lambda^2\delta_x^2e^{im \theta}\right]\hat{u}^{n}=0.
\end{align}
It follows that,
\begin{align*}
&e^{im \theta}
\left[ \rho \left(e^{\theta}+e^{-\theta}\right) +\left(1-2 \rho\right)\right]
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
\notag \\ &
-\left[
2e^{im \theta}
\left[ \rho \left(e^{\theta}+e^{-\theta}\right) +\left(1-2 \rho\right)\right]
+c^2\lambda^2 e^{im \theta}
\left[2\left(e^{\theta}+e^{-\theta}\right) -2\right]\right]
\hat{u}^{n}=0.
\end{align*}
This can be simplified as follows,
\begin{align}
&e^{im \theta}
\left( 2\rho \cos\theta +\left(1-2 \rho\right)\right)
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
\notag \\ &
-\left[
2e^{im \theta}
\left( 2\rho \cos\theta +\left(1-2 \rho\right)\right)
+c^2\lambda^2 e^{im \theta}
\left(2\cos\theta -2\right)
\right]
\hat{u}^{n}=0.
\end{align}
Dividing by $e^{im \theta}$,
\begin{align}\label{eq1A7.05}
&\left( 1-4\rho \sin^2\frac{\theta}{2}\right)
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
-\left[
2\left( 1-4\rho \sin^2\frac{\theta}{2}\right)
%\right. \notag \\ & \left.
-4c^2\lambda^2
\sin^2\frac{\theta}{2}\right]
\hat{u}^{n}=0,
\end{align}
It follow from the fact that $\rho=\frac{1-c^2\lambda^2}{12}$,
\begin{align*}
1-4\rho \sin^2\frac{\theta}{2}=1-\frac{1}{3} \sin^2\frac{\theta}{2}+\frac{c^2\lambda^2}{3} \sin^2\frac{\theta}{2}>0
\qquad \qquad (17)
\end{align*}

Furthermore, set $S=\sin\frac{\theta}{2}$. It follows from (17) that,
\begin{align}
&\left( 1-4\rho S^2\right)
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
%\notag \\ &
-\left[
2\left( 1-4\rho S^2\right)
-4c^2\lambda^2S^2
\right]
\hat{u}^{n}=0.
\end{align}
Hence,
\begin{align}
&\hat{u}^{n+1}
+\left(4c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}
-2\right)
\hat{u}^{n}
+\hat{u}^{n-1}=0.
\end{align}
As a result,
\begin{align}\label{eq1A7.06}
\hat{u}^{n+1}-2\varphi(\theta)\hat{u}^{n}+\hat{u}^{n-1}=0,
\end{align}
where,
$\varphi(\theta)=1-2c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}$.
It follows that,
\begin{align}\label{eq1A7.07}
g^2-2\varphi(\theta)g+1=0,
\qquad \qquad (18)
\end{align}
where $g$ is the amplification factor.

In case that, $|\varphi(\theta)|\geq 1$
then quadratic equation (18) has two real roots and one of which is bigger than 1. This would lead to instability. Therefore,
The necessary and sufficient condition for the scheme to be stable can be considered as follows,
$$|\varphi(\theta)| < 1$$

On one hand, clearly, $S^2=\sin^2\frac{\theta}{2}\leq 1$. Hence, $\varphi(\theta,\theta_y)<  1$.

On the other hand,
\begin{align}\label{eq1A7.08}
1-2c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}> -1.
\end{align}
\begin{align}\label{eq1A7.09}
c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}< 1.
\qquad \qquad (19)
\end{align}
Define,
\begin{align}\label{eq1A7.10}
\chi (S^2)=\frac{S^2}{\left( 1-4\rho S^2\right)}
\end{align}
If $c^2\lambda^2 = 1$ , $c^2\lambda^2 \chi( 1 ) = 1$, the inequality in the equation (19) is not satisfied, the method is not stable. Therefore, $c^2\lambda^2<1$ should be considered. It is easy to see that the numerator of $\chi(S^2)$ is an increasing function, whereas the
denominator of is a decreasing. It follows that, $\chi (S^2)$ takes its maximum at $(1)$. Hence,
\begin{align}\label{eq1A7.11}
c^2\lambda^2 \chi (1)<1.
\end{align}
This means,
\begin{align}\label{eq1A7.12}
\frac{c^2\lambda^2}{\left(\frac{2}{3}+\frac{1}{3}c^2\lambda^2 \right)}< 1.
\end{align}

Therefore, the scheme is stable if
\begin{align}\label{eq1A7.13}
\lambda \max_{x\in \Omega} \{|c(x)|\}=c\lambda < 1.
\end{align}
%\subsection{Dispersion Analysis of the Higher Order Schemes}\label{1A8}
%\% Should be written here.

**Theorem (Convergence of the scheme)**
The numerical scheme is fourth-order in both time and space considering $h=h_x=h_y$.
Without loss of generality, let's assume $f(x,t)=0$ for all $x$ and $t$. Moreover, let $c=\max_{(x)\in \Omega} \{|c(x)|\}$. As a result,
\begin{align}\label{eq1A6.01}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}=
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}.
\end{align}
We know,
\begin{align}\label{eq1A6.02}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}
%&=\left(1+\frac{h^2}{12}\delta_x^2\right)\left(\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}+\frac{1}{12}\frac{d^4}{dt^4} \frac{u_{i}^{n}}{c^2}\tau^2+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4+\mathcal{O}(\tau^6)\right)
%\notag\\ &
&=\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}+\frac{1}{12}\frac{d^4}{dt^4} \frac{u_{i}^{n}}{c^2}\tau^2+\frac{1}{12}\frac{d^4}{dx^2dt^2} \frac{u_{i}^{n}}{c^2}h^2
+\frac{1}{144}\frac{d^6}{dx^2dt^4} \frac{u_{i}^{n}}{c^2}h^2\tau^2\notag\\
&+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2}h^4+\mathcal{O}(h^6+\tau^6).
\end{align}
and
\begin{align}\label{eq1A6.03}
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}
%&=\left(1+\frac{h^2}{12}\delta_x^2\right)\left(\frac{d^2}{dx^2} u_{i}^{n+1}+\frac{1}{12}\frac{d^4}{dx^4} u_{i}^{n+1}h^2+\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n+1}h^4+\mathcal{O}(h^6)\right)
%\notag\\ &
&=\frac{d^2}{dx^2} u_{i}^{n}+\frac{1}{12}\frac{d^4}{dx^4} u_{i}^{n}h^2+\frac{1}{12}\frac{d^4}{dt^2dx^2} u_{i}^{n}\tau^2
+\frac{1}{144}\frac{d^6}{dt^2dx^4} u_{i}^{n}h^2\tau^2\notag\\
&+\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4
+\frac{1}{144}\frac{d^6}{dt^4dx^2} u_{i}^{n}\tau^4+\mathcal{O}(h^6+\tau^6)
\end{align}
Therefore,
\begin{align}\label{eq1A6.04}
&\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}-
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}=
\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}-\frac{d^2}{dx^2} u_{i}^{n}
+\frac{1}{144}\left(\frac{d^4}{dx^2dt^4} \frac{u_{i}^{n}}{c^2}-\frac{d^4}{dt^2dx^4} u_{i}^{n}\right)h^2\tau^2
\notag\\ &
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2}h^4
-\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4
+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4
-\frac{1}{144}\frac{d^6}{dt^4dx^2} u_{i}^{n}\tau^4
%\notag\\ &
+\mathcal{O}(h^6+\tau^6)
\end{align}

The order of method respect to $h$ can be easily obtained if $\tau \rightarrow 0$. In other words,

\begin{align}\label{eq1A6.05}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}-
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}&=
\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}-\frac{d^2}{dx^2} u_{i}^{n}
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2}h^4
\notag\\ &
-\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4
+\mathcal{O}(h^6)
\end{align}

The following relationship between $h$ and $\tau$ is considered to find the order of method to $\tau$,
\begin{equation}\label{eq1A6.06}
h=\mu \tau
\end{equation}
where $\mu$ is a known integer.

Therefore,
\begin{align}\label{eq1A6.07}
&\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}-
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}=
\frac{1}{144}\left(
\frac{d^4}{dx^2dt^4} \frac{u_{i}^{n}}{c^2}
-\frac{d^4}{dt^2dx^4} u_{i}^{n}
\right) \mu^2 \tau^4
\notag\\ &
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2} \mu^4 \tau^4
-\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}\mu^4 \tau^4
+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4
-\frac{1}{144}\frac{d^6}{dt^4dx^2} u_{i}^{n}\tau^4
+\mathcal{O}(\tau^6),
\end{align}
which means that the scheme is 4th order respect to $\tau$.

**Example**: Let's consider the following problem,
\begin{align}
\frac{\partial^2 u}{\partial t^2} &= \mathrm{e}^{-x} \frac{\partial^2 u}{\partial x^2}+{\mathrm{e}}^{t+x}-{\mathrm{e}}^t ,&(x,t)\in (0,1)\times(0,1],\\
u(x,0)&=\mathrm{e}^{x},&x\in [0,1],\\
\frac{\partial }{\partial t}u(x,0) &=  \mathrm{e}^{x},&x\in [0,1],\\
u(0,t) &= \mathrm{e}^{t} &t \in[0,1],\\
u(1,t) &= \mathrm{e}^{t+1},&t \in[0,1].
\end{align}
The exact solution corresponding to the problem can be found as follows,
\begin{equation}
u(x,t)=\mathrm{e}^{x+t}
\end{equation}

The results of analyzing the numerical solution of the problem, using the explicit scheme, is available at the following tables.

In [12]:
def Example01_4thSolver1D(Nx,Nt):
    # Nx = the number of mesh points for space
    # Nt = the number of mesh points for time
    # [a,b] and [T1,T2]
    a = 0.0
    b = 1.0
    T1 = 0.0
    T2 = 1.0
    # Functions
    C2 =lambda x: np.exp(-x)
    g=lambda x: np.exp(x)
    h=lambda x: np.exp(x)
    fa=lambda t: np.exp(t)
    fb=lambda t: np.exp(t+1)
    f=lambda x,t: np.exp(t + x) - np.exp(t)
    Ue=lambda x,t: np.exp(x+t)
    # Additional functions
    gxx=lambda x: np.exp(x);
    gxxxx=lambda x: np.exp(x);
    hxx=lambda x: np.exp(x);
    ft=lambda x,t: np.exp(t + x) - np.exp(t);
    ftt=lambda x,t: np.exp(t + x) - np.exp(t);
    # dx, dt and lambda^2
    dx =(b-a)/Nx
    dt =(T2-T1)/Nt
    l2=(dt/dx)**2
    # discretizing [a,b] and [T1,T2]
    xx=np.linspace(a, b, Nx+1)
    tt=np.linspace(T1, T2, Nt+1)
    # dicretizing c^2(x)
    CC=C2(xx)
    # u initial time value
    u0 = g(xx);
    u=np.zeros(Nx+1, dtype=float)
    u1=np.zeros(Nx+1, dtype=float)
    # Computing the solution at the first time step
    GG= g(xx)+h(xx)*dt+0.5*(C2(xx)*gxx(xx)+f(xx,T1))*(dt**2)+\
    (1/6)*(C2(xx)*hxx(xx)+ft(xx,T1))*(dt**3)+(1/24)*(C2(xx)*gxxxx(xx)+ftt(xx,T1))*(dt**4)
    u1[1:Nx]=GG[1:Nx]
    u1[0]=fa(tt[1])
    u1[Nx]=fb(tt[1])
    # indeces
    mid=list(range(1,Nx))
    sub=list(range(1,Nx-1))
    sup=list(range(2,Nx))
    midp=[i+1 for i in mid]
    midm=[i-1 for i in mid]
    # Matrix A
    A=diags([(1/CC[sub])-l2,(10/CC[mid])+2*l2,(1/CC[sup])-l2], [-1,0,1])
    # Matrix B
    B=diags([(2/CC[sub])+10*l2,(20/CC[mid])-20*l2,(2/CC[sup])+10*l2], [-1,0,1])
    # Matrix C
    C=-A
    # Fa and Fb
    FA= fa(tt)
    FB= fb(tt)
    for n in range(1,Nt):
        FF=((dt**2)/12)*((1/CC[midp])*(f(xx[midp],tt[n+1])+10*f(xx[midp],tt[n])+f(xx[midp],tt[n-1]))+\
        (10/CC[mid])*(f(xx[mid],tt[n+1])+10*f(xx[mid],tt[n])+f(xx[mid],tt[n-1]))+\
        (1/CC[midm])*(f(xx[midm],tt[n+1])+10*f(xx[midm],tt[n])+f(xx[midm],tt[n-1])))
        u=np.zeros(Nx+1, dtype=float)
        bb=np.zeros(len(mid), dtype=float)
        bb[0]=-(1/CC[0]-l2)*FA[n+1]+(2/CC[0]+10*l2)*FA[n]-(1/CC[0]-l2)*FA[n-1]
        bb[Nx-2]=-(1/CC[Nx]-l2)*FB[n+1]+(2/CC[Nx]+10*l2)*FB[n]-(1/CC[Nx]-l2)*FB[n-1]
        u[mid]=spsolve(A,B*u1[mid]+C*u0[mid]+bb+FF)
        u[0]=fa(tt[n+1])
        u[Nx]=fb(tt[n+1])
        u0=u1
        u1=u
        del FF
    # end of the loop
    # Norm
    Norm=LA.norm(u-Ue(xx,tt[Nt]), np.inf)
    return Norm

In [15]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=200;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_4thSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Nt,Norm,Ratio,Log
0,10,200,2.401867e-07,0.000000,0.000000
1,20,200,1.510757e-08,15.898436,3.990813
2,40,200,9.358994e-10,16.142298,4.012774
3,80,200,6.068035e-11,15.423434,3.947052


In [14]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=20;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_4thSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Nt,Norm,Ratio,Log
0,10,20,2.417616e-07,0.000000,0.000000
1,20,40,1.519881e-08,15.906617,3.991555
2,40,80,9.521637e-10,15.962389,3.996605
3,80,160,5.941558e-11,16.025487,4.002296
